In [2]:
import numpy as np

In [7]:
x = np.linspace(-5,  5, 11)
y = np.linspace(-4,  4, 9)
X, Y = np.meshgrid(x,  y)
losses = np.zeros_like(X) 

In [10]:
print(x)
print(y)
print(X)
print(Y)
print(X+Y)

[-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
[-4. -3. -2. -1.  0.  1.  2.  3.  4.]
[[-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]
 [-5. -4. -3. -2. -1.  0.  1.  2.  3.  4.  5.]]
[[-4. -4. -4. -4. -4. -4. -4. -4. -4. -4. -4.]
 [-3. -3. -3. -3. -3. -3. -3. -3. -3. -3. -3.]
 [-2. -2. -2. -2. -2. -2. -2. -2. -2. -2. -2.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.]
 [ 4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.]]
[[-9. -8. -7. -6. -5. -4. -3. -2. -1.  0.  1.]
 [-8. -7. -6. -5. -4.

In [ ]:
import numpy as np 
from matplotlib import pyplot as plt
import torch 
import torch.nn  as nn 
import torchvision 
from tqdm import tqdm 
from matplotlib.colors  import LogNorm 
from mpl_toolkits.mplot3d  import Axes3D 
from sklearn.decomposition  import PCA 
 
# ====================== 1. 数据准备 ======================
def create_2d_mnist():
    """将MNIST压缩到二维特征空间"""
    mnist = torchvision.datasets.MNIST(root='./data',  train=True, download=True)
    pca = PCA(n_components=2)
    features = pca.fit_transform(mnist.data.numpy().reshape(-1,  28*28))
    y = torch.zeros(len(mnist.targets),  10)  # 转为one-hot 
    y[range(len(mnist.targets)),  mnist.targets]  = 1 
    return torch.FloatTensor(features), y 
 
X, y = create_2d_mnist()
X = (X - X.mean())  / X.std()   # 标准化 
 
# ====================== 2. 定义模型 ======================
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1  = nn.Linear(2, 10, bias=False)
        self.fc2  = nn.Linear(10, 10, bias=False)
        
    def forward(self, x):
        return self.fc2(torch.relu(self.fc1(x))) 
 
# ====================== 3. 损失地形计算 ======================
def compute_loss_landscape(model, criterion):
    """计算二维参数空间的损失曲面"""
    w1 = np.linspace(-3,  3, 100)  # 参数1范围 
    w2 = np.linspace(-3,  3, 100)  # 参数2范围 
    W1, W2 = np.meshgrid(w1,  w2)
    losses = np.zeros_like(W1) 
    
    original_weights = model.fc1.weight.data.clone() 
    for i in tqdm(range(100), desc="计算地形"): 
        for j in range(100):
            with torch.no_grad(): 
                model.fc1.weight[0,0]  = W1[i,j]
                model.fc1.weight[0,1]  = W2[i,j]
            losses[i,j] = criterion(model(X), y).item()
    model.fc1.weight.data  = original_weights  # 恢复权重 
    return W1, W2, losses 
 
# ====================== 4. 轨迹跟踪器 ======================
class TrajectoryTracker:
    def __init__(self, model):
        self.positions  = []
        self.model  = model 
        
    def track(self):
        with torch.no_grad(): 
            w1 = self.model.fc1.weight[0,0].item() 
            w2 = self.model.fc1.weight[0,1].item() 
            self.positions.append([w1,  w2])
 
# ====================== 5. 优化器比较 ======================
def compare_optimizers():
    """测试不同优化器的轨迹"""
    model = SimpleModel()
    criterion = nn.MSELoss()
    
    # 定义优化器（此处假设PUGD已实现）
    optimizers = {
        "SGD": torch.optim.SGD(model.parameters(),  lr=0.1),
        "Adam": torch.optim.Adam(model.parameters(),  lr=0.01),
        "PUGD": PUGD(model.parameters(),  lr=0.1)  # 自定义优化器 
    }
    
    trajectories = {}
    for name, opt in optimizers.items(): 
        # 重置模型权重 
        for param in model.parameters(): 
            nn.init.normal_(param,  mean=0, std=0.1)
            
        tracker = TrajectoryTracker(model)
        for epoch in tqdm(range(50), desc=f"{name}优化"):
            opt.zero_grad() 
            loss = criterion(model(X), y)
            loss.backward() 
            opt.step() 
            tracker.track() 
        trajectories[name] = np.array(tracker.positions) 
    return trajectories 
 
# ====================== 6. 可视化 ======================
def visualize(W1, W2, losses, trajectories):
    """绘制3D地形和2D等高线图"""
    plt.figure(figsize=(18,  6))
    
    # 3D曲面图 
    ax1 = plt.subplot(121,  projection='3d')
    surf = ax1.plot_surface(W1,  W2, losses, cmap='viridis', 
                          alpha=0.6, antialiased=True)
    ax1.set_zscale('log') 
    ax1.set_title("3D  Loss Landscape")
    
    # 2D等高线图 
    ax2 = plt.subplot(122) 
    contour = ax2.contour(W1,  W2, losses, levels=15, norm=LogNorm(), cmap='viridis')
    plt.colorbar(contour,  label='Loss Value')
    ax2.set_title("Optimization  Trajectories")
    
    # 绘制轨迹 
    colors = {'SGD': 'red', 'Adam': 'blue', 'PUGD': 'green'}
    for name, path in trajectories.items(): 
        ax1.plot(path[:,0],  path[:,1], losses.min(),  
                c=colors[name], lw=2, label=name)
        ax2.plot(path[:,0],  path[:,1], 'o-', 
                c=colors[name], markevery=5, label=name)
    
    plt.legend() 
    plt.tight_layout() 
    plt.savefig('optim_comparison.png',  dpi=300, bbox_inches='tight')
 
# ====================== 主执行流程 ======================
if __name__ == "__main__":
    # 计算损失地形 
    model = SimpleModel()
    criterion = nn.MSELoss()
    W1, W2, losses = compute_loss_landscape(model, criterion)
    
    # 比较优化器 
    trajectories = compare_optimizers()
    
    # 可视化结果 
    visualize(W1, W2, losses, trajectories)
    plt.show() 

In [ ]:
import numpy as np 
import matplotlib.pyplot  as plt 
from mpl_toolkits.mplot3d  import Axes3D 
from matplotlib.animation  import FuncAnimation 
from matplotlib.colors  import ListedColormap 
import torch 
 
# ========== 1. 模拟优化过程 ==========
def simulate_optimization():
    """生成模拟优化轨迹和损失历史"""
    # 创建虚拟参数空间网格 
    x = np.linspace(-5,  5, 100)
    y = np.linspace(-3,  3, 100)
    X, Y = np.meshgrid(x,  y)
    
    # 定义损失曲面 (Rosenbrock函数)
    Z = (1 - X)**2 + 100 * (Y - X**2)**2 
    
    # 模拟优化轨迹 (SGD路径)
    trajectory = []
    current_pos = np.array([-4.0,  3.0])
    lr = 0.01 
    for step in range(200):
        grad_x = -2*(1-current_pos[0]) - 400*current_pos[0]*(current_pos[1]-current_pos[0]**2)
        grad_y = 200*(current_pos[1] - current_pos[0]**2)
        current_pos -= lr * np.array([grad_x,  grad_y])
        trajectory.append(current_pos.copy()) 
    
    return X, Y, Z, np.array(trajectory) 
 
# ========== 2. 创建3D透明环境 ==========
def create_3d_environment(X, Y, Z, trajectory):
    fig = plt.figure(figsize=(12,  8))
    ax = fig.add_subplot(111,  projection='3d')
    
    # 绘制半透明损失曲面 
    surf = ax.plot_surface(X,  Y, Z, cmap='viridis', 
                         alpha=0.4, antialiased=True,
                         rstride=2, cstride=2)
    
    # 添加等高线投影 
    ax.contour(X,  Y, Z, 10, zdir='z', offset=Z.min(),  
              cmap='viridis', alpha=0.3)
    
    # 初始化轨迹线 
    line, = ax.plot([],  [], [], 'r-', lw=2)
    point, = ax.plot([],  [], [], 'ro', markersize=8)
    
    # 动态更新函数 
    def update(frame):
        # 更新轨迹线 
        line.set_data(trajectory[:frame,  0], trajectory[:frame, 1])
        line.set_3d_properties(Z.min()) 
        
        # 更新当前位置点 
        x, y = trajectory[frame]
        z = (1 - x)**2 + 100 * (y - x**2)**2 
        point.set_data([x],  [y])
        point.set_3d_properties([z]) 
        
        # 动态调整视角 
        ax.view_init(elev=30,  azim=frame/2)
        
        # 透明度渐变效果 
        surf.set_alpha(0.2  + 0.6 * (1 - frame/200))
        
        return line, point, surf 
    
    # 创建动画 
    ani = FuncAnimation(fig, update, frames=len(trajectory), 
                       interval=50, blit=True)
    
    ax.set_xlabel('Parameter  1')
    ax.set_ylabel('Parameter  2')
    ax.set_zlabel('Loss  Value')
    plt.tight_layout() 
    return ani 
 
# ========== 3. 执行与保存 ==========
if __name__ == "__main__":
    X, Y, Z, trajectory = simulate_optimization()
    ani = create_3d_environment(X, Y, Z, trajectory)
    
    # 保存为GIF（需安装pillow）
    ani.save('3d_optimization.gif',  writer='pillow', fps=20, dpi=150)
    
    # 或直接显示 
    plt.show() 